In [13]:
#Import Libraries
import pandas as pd
import numpy as np
# Loading Data
df = pd.read_csv('/Users/AribM/Desktop/MF DA/Mutual_Funds.csv')
# Filtering and Cleaning Data
df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)
df=df[df['Date'] >= '2018-07-23'] # Analysing Data for 5 year period, i.e, 2018-07-23 to 2023-07-23

In [14]:
df['min_date'] = df.groupby(by='Scheme_Code')['Date'].transform('min') # Getting the earliest date of fund data
df['max_date'] = df.groupby(by='Scheme_Code')['Date'].transform('max') # Getting the latest date of fund data

In [15]:
df = df.sort_values(by=['Scheme_Code','Date']).reset_index(drop=True) # Sorting by Scheme_Code and Date

In [16]:
df=df[df['max_date'] >= '2023-01-01'] #Removing Mutual Funds whose data has not been recieved after 2022
df['no_of_days']=(df['max_date']-df['min_date']).dt.days # Calculating the number of days of data for the given fund
df=df[df['no_of_days'] >= 365] # Filtering out MFs whose data is available for less than 1 year

In [17]:
df = df[df['NAV'] != 0] # Removing 0 NAV values
df.dropna(subset='NAV',inplace=True) # Removing NaN NAV values for consistency and precision 

In [18]:
df['returns'] = df.groupby('Scheme_Code')['NAV'].pct_change() * 100  # Getting the daily returns of the Funds
df['returns'] = df['returns'].fillna(0) # Filling NaN values in returns column with 0 
df['std_dev'] = df.groupby('Scheme_Code')['returns'].transform('std') # Calculating fund wise Standard Deviation 


In [19]:
df['average_fund_return'] = df.groupby('Scheme_Code')['returns'].transform('mean') * 252 # Annualizing the fund returns

In [20]:
# Computing Values for calculating total Active period between the analysis period
df['scheme_iv'] = df.groupby('Scheme_Code')['NAV'].transform('first')
df['scheme_fv'] = df.groupby('Scheme_Code')['NAV'].transform('last')
df_active_years = df.groupby('Scheme_Code').agg(
    scheme_min_date=('min_date', 'min'),
    scheme_max_date=('max_date', 'max')
).reset_index()

df_active_years['active_years'] = (df_active_years['scheme_max_date'] - df_active_years['scheme_min_date']).dt.days / 365
# Merge back with the original DataFrame
df = pd.merge(df,df_active_years,on='Scheme_Code', how='left')
df = df[(df['scheme_iv'] > 0) & (df['active_years'] > 0)] # Prevent division by 0
df['CAGR'] = ((df['scheme_fv']/df['scheme_iv']) ** (1/df['active_years']) - 1 )* 100 # Calculating Fund wise CAGR

In [21]:
df.drop(['scheme_fv','scheme_iv'],axis=1,inplace=True) # Dropping final and initial NAV values of funds from main dataframe

In [22]:
# Beta Calculation
# Since Data does not have Beta Values nor benchmark details, we will have to estimate beta for different types of schemes
# Using std dev method of scheme and market for equity schemes and CAGR approximation for other schemes since standard deviation data might be unreliable

market_std = 0.18 # Market standard deviation for equity schemes
market_cagr = 0.08 # Market CAGR
r_f_r = 0.04 # risk free rate

df['Scheme_Category'] = df['Scheme_Category'].astype(str)

def compute_beta(row):
     if "Equity" in row['Scheme_Category']:  
        # Equity Schemes: Beta using Standard Deviation
        return row['std_dev'] / market_std if row['std_dev'] > 0 else None
     else:
        # Non-Equity Schemes: Beta using CAGR approximation
        return (row['CAGR'] - r_f_r) / (market_cagr - r_f_r) if row['CAGR'] > r_f_r else None
     
df['Estimated_Beta'] = df.apply(compute_beta,axis=1)
df['Estimated_Beta'] = df['Estimated_Beta'].clip(-5, 5) # Unrealistic Beta Values rounded to prevent skewing of Beta values

In [23]:
# assuming average daily market return of 8.0%
# assuming Beta as 1 since we don't have actual values
# assuming risk free rate from govt. bond yields of 4%
# calculating alpha
def alpha_values(row) -> float:
    alpha_val = row['average_fund_return'] - (4 + (row['Estimated_Beta'] * (8 - 4)))
    return alpha_val

In [24]:
df['alpha'] = df.apply(alpha_values,axis=1)
df['alpha'] = df['alpha'].clip(lower=-10, upper=10) # Clipping to avoid rare / unreasonable alpha values

In [ ]:
# Calculating Sortino Ratio

# assuming the risk free rate of 6%
risk_free_rate = 6

def calculate_sortino(group):
    scheme_avg_return = group['returns'].mean() * 252 # Annualizing returns 
    downside_returns = group[group['returns'] < risk_free_rate]['returns'] - risk_free_rate # computing downside returns
    downside_dev = downside_returns.std() * np.sqrt(252) # Calculating downside deviation of funds and annualizing 

    if downside_dev == 0 or np.isnan(downside_dev): # if no negative values then return NaN
        return np.nan
    
    return (scheme_avg_return - risk_free_rate) / downside_dev # Sortino formula

df_sortino = df.groupby('Scheme_Code').apply(calculate_sortino).reset_index(name="Sortino_Ratio")
# Created a dataframe containing sortino values 

In [26]:
df = df.merge(df_sortino,on = 'Scheme_Code',how='left') # Merging sortino df to main df

In [27]:
# Calculating Sharpe Ratio
rfr = 0.06 # assuming 6% annual risk free rate
df_sharpe = df.groupby('Scheme_Code')['returns'].agg(['mean','std']).reset_index()
df_sharpe['mean_annual'] = df_sharpe['mean'] * 252
df_sharpe['std_annual'] = df_sharpe['std'] * (252 ** 0.5)

df_sharpe['Sharpe_annual'] = (df_sharpe['mean_annual'] - rfr) / df_sharpe['std_annual'] # Annualizing Sharpe values
df_sharpe.fillna({'Sharpe_annual': 0}, inplace=True)  # replacing Nan values by 0
df = df.merge(df_sharpe[['Scheme_Code','Sharpe_annual']],on='Scheme_Code',how='left') # merge back to main df

In [28]:
# Creating a shorter dataframe with relevant data and all calculated fields
df_processed = df.drop(['scheme_min_date','scheme_max_date','min_date','max_date','Date','NAV','no_of_days','returns','average_fund_return'],axis = 1).copy()
df_processed['active_years'] = df_processed['active_years'].round()
cols_to_round = ['std_dev','alpha','CAGR']
df_processed[cols_to_round] = df_processed[cols_to_round].round(3) # rounding off column values upto 3 decimals
df_processed.drop_duplicates(subset='Scheme_Code',inplace=True) # Only 1 row per scheme
df_processed = df_processed.reset_index()
df_processed.drop(columns=['index'],inplace=True)
df_processed.index.name = "S.No" # Renaming Index Name

In [29]:
# Dropping Nan values if in any 3 columns
df_processed.dropna(subset='alpha',inplace=True) 
df_processed.dropna(subset='Estimated_Beta',inplace=True)
df_processed.dropna(subset='CAGR',inplace=True)
print(df_processed) # printing to check the new dataframe

                             Fund_House          Scheme_Type  \
S.No                                                           
0     Aditya Birla Sun Life Mutual Fund   Open Ended Schemes   
1     Aditya Birla Sun Life Mutual Fund   Open Ended Schemes   
2     Aditya Birla Sun Life Mutual Fund   Open Ended Schemes   
3     Aditya Birla Sun Life Mutual Fund   Open Ended Schemes   
5     Aditya Birla Sun Life Mutual Fund  360 ONE Mutual Fund   
...                                 ...                  ...   
6198  Aditya Birla Sun Life Mutual Fund   Open Ended Schemes   
6199            Mirae Asset Mutual Fund   Open Ended Schemes   
6200            Mirae Asset Mutual Fund   Open Ended Schemes   
6201            Mirae Asset Mutual Fund   Open Ended Schemes   
6202            Mirae Asset Mutual Fund   Open Ended Schemes   

                                 Scheme_Category  Scheme_Code  \
S.No                                                            
0           Equity Scheme - Large & M

In [31]:
# Getting a filtered DataFrame with good performing funds
df_filtered = df_processed[
    (df_processed['Estimated_Beta'].between(-3, 3)) &   # Remove extremely volatile funds
    (df_processed['Sharpe_annual'] > 0) &               # Keep funds with positive Sharpe
    (df_processed['Sortino_Ratio'] > 0)                 # Keep funds with decent risk-adjusted returns
]

In [32]:
# Creating DataFrames according to scheme types for efficiency and analysis
df_equity = df_processed[df_processed['Scheme_Category'].str.contains("Equity",case=False,na=False)]
df_debt = df_processed[df_processed['Scheme_Category'].str.contains("Debt",case=False,na=False)]
df_hybrid = df_processed[df_processed['Scheme_Category'].str.contains("Hybrid",case=False,na=False)]
df_other = df_processed[df_processed['Scheme_Category'].str.contains("Other",case=False,na=False)]

In [33]:
df_top_200_CAGR = df_filtered.nlargest(200,'CAGR') # Getting top funds from the filtered funds according to CAGR
df_top_ratio_mfs = df_filtered.sort_values(['Sharpe_annual', 'Sortino_Ratio'], ascending=[False, False]).head(200)
# Df for top sharpe and sortino ratio funds in filtered funds 

In [34]:
! pip install openpyxl
file_path = "processed_data.xlsx"

# Write all DataFrames to the same file in different sheets
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    df_processed.to_excel(writer, sheet_name="Processed_Data", index=False)
    df_equity.to_excel(writer, sheet_name="Equity_Schemes", index=False)
    df_debt.to_excel(writer, sheet_name="Debt_Schemes", index=False)
    df_hybrid.to_excel(writer, sheet_name="Hybrid_Schemes", index=False)
    df_other.to_excel(writer, sheet_name="Other_Schemes", index=False)

1860.64s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [36]:
df.drop(['scheme_min_date','scheme_max_date'],axis=1,inplace=True)
df['returns'].fillna(0,inplace=True)
df.to_csv("detailed_data.csv",index=False)

/var/folders/8s/_3bwkhvn5hn6yrtnxm4mtfqw0000gn/T/ipykernel_1571/2229188980.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['returns'].fillna(0,inplace=True)
